In [1]:
#Leduc hold'em is a simplified poker game in which each player gets 1 card
#Each player automatically puts 1 chip into the pot to begin the hand (called an ante)
#This is followed by the first round (called preflop) of betting
#In this round the bet size is 2 and there is a maximum of 2 bets (i.e. 1 bet and 1 raise)
#If neither player folds, the hand goes on to the second round (the flop)
#In this round the bet size is 4 and again there is a maximum of 2 bets
#If neither player folds, the pot is given to the best hand at showdown
#The best hand is either a pair if the community card matches a player's private card
#Or simply the highest private card if neither player matches the community card

import random

def validbets(history,betsize):   
    betoptions = ['f','k','c','b','r'] #fold check call bet raise
    
    #logical checks to see when certain betoptions are not allowed (one invalidation line for each)
    #possible previous history in this round is {'', 'k', 'b', 'r'} (empty, check, bet, raise)
    #note that fold and call will never be the previous action here in this function because 
    #these trigger the end of a hand or end of a round
    if history[-1:] == 'b' or history[-1:] == 'r': #previous action bet or raise
        betoptions[3] = False #Bet invalid -- Can only bet at the start of a round or after a check
        betoptions[1] = False #Check invalid -- Can only check at the start of a round or after a check
        
    if history[-1:] != 'b':
        betoptions[4] = False #Raise invalid -- Can only raise after a bet (and only one raise per round)
        
    if history[-1:] == 'k' or history[-1:] == '': #previous action check or none
        betoptions[0] = False #Fold invalid -- Can only fold after a bet or a raise
        betoptions[2] = False #Call invalid -- Can only call after a bet or a raise
    
    #print the bet options that are available
    if betoptions[0]: print('FOLD: f')
    if betoptions[1]: print('CHECK: k')
    if betoptions[2]: print('CALL {}: c'.format(betsize))
    if betoptions[3]: print('BET {}: b'.format(betsize))
    if betoptions[4]: print('RAISE {}: r'.format(betsize*2))
    
    #note that fold and check are never both options at the same time
    #note that bet and raise are never both options at the same time
    return makebet(betoptions)
    
def makebet(betoptions):
    betinput = input()
    if betinput in betoptions:
        return betinput
    else:
        print('\nInvalid bet made, try again')
        return makebet(betoptions)

def playpreflop(p0card,p1card,flopcard,r1history,r2history,dealer,profit0):
    betsize = 2 #bet size is fixed = 2 for this round
    print(r1history, r2history)
    
    #determine which player is acting and the size of the pot
    plays = len(r1history) #number of actions in preflop round
    actingplayer = (plays+dealer) % 2 #start with dealer and rotate each action
    #in 1v1 poker games with multiple rounds, the dealer acts first in the first round and last in other rounds 
    opponentplayer = 1 - actingplayer #players are 0 and 1
    pot = 2.+betsize*r1history.count('b')+betsize*r1history.count('c')+2*betsize*r1history.count('r')

    #check to see if the game is over because last player folded
    if r1history[-1:] == 'f': 
        profit = (pot - betsize)/2 #subtract uncalled bet of 2 and divide by 2 (since profit is opponent's half)
        if actingplayer == 0: #the other player was the one who folded
            profit0 +=profit #keep track of profit from player 0 perspective (player 1 is just opposite)
        else:
            profit0 -=profit
        print('End of game! Player {} wins pot of {} (profits {})\n'.format(actingplayer,pot,profit))
        main(1-dealer,profit0) #next hand, swap dealer
    
    #check to see if moving to the next round because there were 2 checks in a row or a call
    if r1history[-2:] == 'kk' or r1history[-1:] == 'c': 
        print('\nDealing flop card: {}'.format(flopcard)) #deal flop card for 2nd betting round
        playflop(p0card,p1card,flopcard,r1history,r2history,dealer,profit0) #get flop action
    
    #game/round not over, continue with next action in this round
    print('\nPot size: {}'.format(pot))       
    print('Player {} turn to act'.format(actingplayer))
    bet = validbets(r1history,betsize) #checks which bet sizes are possible then asks for user input
    playpreflop(p0card,p1card,flopcard,r1history+bet,r2history,dealer,profit0) #next preflop action

    
def playflop(p0card,p1card,flopcard,r1history,r2history,dealer,profit0):
    betsize = 4 #bet size is fixed = 4 for this round
    print(r1history, r2history)
    
    #determine which player is acting and the size of the pot
    plays = len(r2history) #number of actions in flop round
    opponentplayer = (plays+dealer) % 2 #start with non-dealer and rotate each action
    actingplayer = 1 - opponentplayer #players are 0 and 1
    pot = 2+(betsize/2)*r1history.count('b')+(betsize/2)*r1history.count('c')+2*(betsize/2)*r1history.count('r') \
        +betsize*r2history.count('b')+betsize*r2history.count('c')+2*betsize*r2history.count('r')

    #check to see if the game is over because the last player folded
    if r2history[-1:] == 'f': 
        profit = (pot - betsize)/2 #subtract uncalled bet of 4 and divide by 2 (since profit is opponent's half)
        if actingplayer == 0: #the other player was the one who folded
            profit0 +=profit #keep track of profit from player 0 perspective (player 1 is just opposite)
        else:
            profit0 -=profit
        print('End of game! Player {} wins pot of {} (profits {})\n'.format(actingplayer,pot,profit))
        main(dealer-1, profit0) #next hand, swap dealer
    
    #2 checks in a row or a bet/raise and a call >>>> hand is over and goes to showdown  
    if r2history[-2:] == 'kk' or r2history[-1:] == 'c': 
        if p0card == flopcard: #automatically win with a pair if card matches shared card
            winner = 0
        elif p1card == flopcard: #automatically win with a pair if card matches shared card
            winner = 1
        elif p1card > p0card: #no one has the pair, compare the cards against each other
            winner = 1
        elif p0card > p1card: #no one has the pair, compare the cards against each other
            winner = 0
        else: #tie game
            winninghand = [p0card, flopcard]
            print('Tie game! Pot size {} and both players had {}\n'.format(pot,winninghand))
            main(1-dealer,profit0)
        profit = pot/2 #profit is the opponent's half of the pot
        if winner == 0: #keep track of profit from player 0 perspective (player 1 is just opposite)
            profit0 +=profit
            winninghand = [p0card, flopcard]
        else:
            profit0 -=profit
            winninghand = [p1card, flopcard]
        print('End of game! Player {} wins pot of {} with {} (profits {})\n'.format(winner,pot,winninghand,profit))
        main(1-dealer,profit0) #next hand, swap dealer
    
    #game/round not over, continue with next action in this round 
    print('\nPot size: {}'.format(pot))
    print('Player {} turn to act'.format(actingplayer))
    bet = validbets(r2history,betsize) #checks which bet sizes are possible then asks for user input
    playflop(p0card,p1card,flopcard,r1history,r2history+bet,dealer,profit0) #next flop action
    

def main(dealer, profit0):
    #here we shuffle the deck and define a card for
    #each player and the shared community card (only
    #relevant if the players get to that betting round)
    cards = [1,1,2,2,3,3] #define 6 card poker deck (can think of it as 1=Jack, 2=Queen, 3=King)
    cards_shuffled = random.shuffle(cards) #shuffle
    if dealer == 0: #deal cards according to the standard order
        p1card = cards[0]
        p0card = cards[1]
    else:
        p0card = cards[0]
        p1card = cards[1]
    flopcard = cards[2] #the shared community card is the next card in the deck
    print('=================\nStarting new game\n=================')
    print('Player 0 profit: {}'.format(profit0))
    print('Player 1 profit: {}'.format(-profit0))
    print ('')
    print('Player 0 card: {}'.format(p0card))
    print('Player 1 card: {}'.format(p1card))
    print ('')
    playpreflop(p0card,p1card,flopcard,'','',dealer,profit0) #each player puts 1 into the pot automatically
    
main(0,0) #define dealer as player 0 for the first game 
#(dealer is the term for the player who "deals" and acts first preflop and acts last in other rounds)

Starting new game
Player 0 profit: 0
Player 1 profit: 0

Player 0 card: 3
Player 1 card: 3

 

Pot size: 2.0
Player 0 turn to act
CHECK: k
BET 2: b
b
b 

Pot size: 4.0
Player 1 turn to act
FOLD: f
CALL 2: c
RAISE 4: r
c
bc 

Dealing flop card: 1
bc 

Pot size: 6.0
Player 1 turn to act
CHECK: k
BET 4: b
k
bc k

Pot size: 6.0
Player 0 turn to act
CHECK: k
BET 4: b
k
bc kk
Tie game! Pot size 6.0 and both players had [3, 1]

Starting new game
Player 0 profit: 0
Player 1 profit: 0

Player 0 card: 2
Player 1 card: 1

 

Pot size: 2.0
Player 1 turn to act
CHECK: k
BET 2: b
b
b 

Pot size: 4.0
Player 0 turn to act
FOLD: f
CALL 2: c
RAISE 4: r
c
bc 

Dealing flop card: 3
bc 

Pot size: 6.0
Player 0 turn to act
CHECK: k
BET 4: b


KeyboardInterrupt: 

In [3]:
i = range(5)
print(i)

range(0, 5)


In [2]:
i = ['f', 'k', 'c']

In [3]:
i[2]

'c'

In [11]:
a = i[0:2]

In [12]:
b = ['f', 'k']

In [13]:
a == b

True

In [14]:
i[-1]

'c'

In [15]:
bet_options = ['f', 'k', 'c', 'b', 'r']
bet_options[2] = False

In [16]:
bet_options

['f', 'k', False, 'b', 'r']

In [17]:
def test(stone):
    print(stone)
    
test(bet_options)

['f', 'k', False, 'b', 'r']


In [21]:
random.choice(bet_options)

False